## Import the necessary basic libraries

In [1]:
import numpy as np
import pandas as pd
import csv
import re
import requests
from bs4 import BeautifulSoup

## Cleaning the Oscar Award dataset

### Obtaining the dataset

In [2]:
oscarData = pd.read_csv('the_oscar_award.csv')
oscarData.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


###### Check the vital statistics of the dataset using the type and shape attributes:

In [3]:
print("Data type : ", type(oscarData))
print("Data dims : ", oscarData.shape)

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (10395, 7)


###### Check the variables (and their types) in the dataset using the dtypes attribute.

In [4]:
print(oscarData.dtypes)

year_film         int64
year_ceremony     int64
ceremony          int64
category         object
name             object
film             object
winner             bool
dtype: object


###### Information about the variables:

In [5]:
oscarData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10395 entries, 0 to 10394
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10395 non-null  int64 
 1   year_ceremony  10395 non-null  int64 
 2   ceremony       10395 non-null  int64 
 3   category       10395 non-null  object
 4   name           10395 non-null  object
 5   film           10091 non-null  object
 6   winner         10395 non-null  bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 497.5+ KB


- We can see that there are missing values for the film category

In [6]:
oscarData['film'].isnull().value_counts()

False    10091
True       304
Name: film, dtype: int64

- There are a total of 304 missing values for the film category

- The categories with missing values for the film category

In [7]:
print(oscarData[oscarData['film'].isnull()]['category'].unique())

['ENGINEERING EFFECTS' 'WRITING (Title Writing)' 'SPECIAL AWARD'
 'SOUND RECORDING' 'ASSISTANT DIRECTOR'
 'IRVING G. THALBERG MEMORIAL AWARD' 'SPECIAL FOREIGN LANGUAGE FILM AWARD'
 'HONORARY FOREIGN LANGUAGE FILM AWARD' 'HONORARY AWARD'
 'JEAN HERSHOLT HUMANITARIAN AWARD' 'SPECIAL ACHIEVEMENT AWARD']


In [8]:
print(oscarData[oscarData['film'].isnull()]['category'].nunique())

11


There are 11 categories in total.

In [9]:
oscarData[oscarData['film'].isnull()].head()

,year_film,year_ceremony,ceremony,category,name,film,winner
16,1927,1928,1,ENGINEERING EFFECTS,Ralph Hammeras,NaN,False
18,1927,1928,1,ENGINEERING EFFECTS,Nugent Slaughter,NaN,False
31,1927,1928,1,WRITING (Title Writing),Joseph Farnham,NaN,True
32,1927,1928,1,WRITING (Title Writing),"George Marion, Jr.",NaN,False
33,1927,1928,1,SPECIAL AWARD,Warner Bros.,NaN,True


### Cleaning the dataset:

- There are a few awards that are not related to the movie itself, hence we should drop these lines

In [10]:
dropRows = oscarData[
    (oscarData['category'] == 'HONORARY AWARD') |
    (oscarData['category'] == 'SPECIAL AWARD') |
    (oscarData['category'] == 'IRVING G. THALBERG MEMORIAL AWARD') |
    (oscarData['category'] == 'JEAN HERSHOLT HUMANITARIAN AWARD') |
    (oscarData['category'] == 'SPECIAL ACHIEVEMENT AWARD')
].index

oscarData = oscarData.drop(dropRows)

In [11]:
oscarData = oscarData.dropna(how='all')

In [12]:
oscarData.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


- These are the remaining categories with empty film names:

In [13]:
print(oscarData[oscarData['film'].isnull()]['category'].unique())
oscarData[oscarData['film'].isnull()].head()

['ENGINEERING EFFECTS' 'WRITING (Title Writing)' 'SOUND RECORDING'
 'ASSISTANT DIRECTOR' 'SPECIAL FOREIGN LANGUAGE FILM AWARD'
 'HONORARY FOREIGN LANGUAGE FILM AWARD']


,year_film,year_ceremony,ceremony,category,name,film,winner
16,1927,1928,1,ENGINEERING EFFECTS,Ralph Hammeras,NaN,False
18,1927,1928,1,ENGINEERING EFFECTS,Nugent Slaughter,NaN,False
31,1927,1928,1,WRITING (Title Writing),Joseph Farnham,NaN,True
32,1927,1928,1,WRITING (Title Writing),"George Marion, Jr.",NaN,False
145,1930,1931,4,SOUND RECORDING,Samuel Goldwyn - United Artists Studio Sound D...,NaN,False


- Through observation, we can see that there are 2 categories which have the film name in the 'name' column instead of the 'film' column. These categories are the "SPECIAL FOREIGN LANGUAGE FILM AWARD" and the "HONORARY FOREIGN LANGUAGE FILM AWARD".

In [14]:
for row in oscarData [
    (oscarData['category'] == 'SPECIAL FOREIGN LANGUAGE FILM AWARD') |
    (oscarData['category'] == 'HONORARY FOREIGN LANGUAGE FILM AWARD')
].iterrows():
    
    print(row[1][4])

 Monsieur Vincent - voted by the Academy Board of Governors as the most outstanding foreign language film released in the United States during 1948.
 The Bicycle Thief - voted by the Academy Board of Governors as the most outstanding foreign language film released in the United States during 1949.
 The Walls of Malapaga - voted by the Board of Governors as the most outstanding foreign language film released in the United States in 1950.
 Rashomon - voted by the Board of Governors as the most outstanding foreign language film released in the United States during 1951.
Forbidden Games - Best Foreign Language Film first released in the United States during 1952.
 Gate of Hell - Best Foreign Language Film first released in the United States during 1954.
 Samurai


- We should move the film name to the 'film' column for each of these categories

In [15]:
oscarData['film'] = oscarData.apply(
    lambda x: x['name'].split('-')[0] if x['category'] in (
        'SPECIAL FOREIGN LANGUAGE FILM AWARD', 'HONORARY FOREIGN LANGUAGE FILM AWARD'
    )
    else x['film'],
    axis=1
)

- These are the remaining categories with empty film names:

In [16]:
print(oscarData[oscarData['film'].isnull()]['category'].unique())
oscarData[oscarData['film'].isnull()]

['ENGINEERING EFFECTS' 'WRITING (Title Writing)' 'SOUND RECORDING'
 'ASSISTANT DIRECTOR']


,year_film,year_ceremony,ceremony,category,name,film,winner
16,1927,1928,1,ENGINEERING EFFECTS,Ralph Hammeras,NaN,False
18,1927,1928,1,ENGINEERING EFFECTS,Nugent Slaughter,NaN,False
31,1927,1928,1,WRITING (Title Writing),Joseph Farnham,NaN,True
32,1927,1928,1,WRITING (Title Writing),"George Marion, Jr.",NaN,False
145,1930,1931,4,SOUND RECORDING,Samuel Goldwyn - United Artists Studio Sound D...,NaN,False
146,1930,1931,4,SOUND RECORDING,Metro-Goldwyn-Mayer Studio Sound Department,NaN,False
147,1930,1931,4,SOUND RECORDING,Paramount Publix Studio Sound Department,NaN,True
148,1930,1931,4,SOUND RECORDING,RKO Radio Studio Sound Department,NaN,False
192,1931,1932,5,SOUND RECORDING,Metro-Goldwyn-Mayer Studio Sound Department,NaN,False
193,1931,1932,5,SOUND RECORDING,Paramount Publix Studio Sound Department,NaN,True


- Upon conducting some research, we found that these 4 remaining categories have since been removed. Hence, we will not be considering the award winners for these categories and will be dropping them from our oscarData dataset.

In [17]:
dropRows = oscarData[
    (
        (oscarData['category'] == 'ENGINEERING EFFECTS') |
        (oscarData['category'] == 'WRITING (Title Writing)') |
        (oscarData['category'] == 'SOUND RECORDING') |
        (oscarData['category'] == 'ASSISTANT DIRECTOR')
    )
].index

oscarData = oscarData.drop(dropRows)

In [18]:
oscarData = oscarData.dropna(how = 'all')
oscarData.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


Let's take a look at our cleaned dataset. The 'film' category should now have the same number of data points as the other categories.

In [19]:
oscarData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9892 entries, 0 to 10390
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      9892 non-null   int64 
 1   year_ceremony  9892 non-null   int64 
 2   ceremony       9892 non-null   int64 
 3   category       9892 non-null   object
 4   name           9892 non-null   object
 5   film           9892 non-null   object
 6   winner         9892 non-null   bool  
dtypes: bool(1), int64(3), object(3)
memory usage: 550.6+ KB


### Calculate the number of wins for each film.
(Remove duplicate films as we are only interested in the film and its number of wins at the Oscar Awards.)

###### Create a new column for the number of wins:

In [20]:
oscarData.insert(7, "Number of wins", 0)

In [21]:
oscarData.head()

,year_film,year_ceremony,ceremony,category,name,film,winner,Number of wins
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False,0
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True,0
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False,0
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True,0
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,0


###### Calculating and adding the number of wins for each film into the oscarData dataset

In [22]:
title = ""
year = 0
wins = 0
enter = 1

for i in range(0, 9893):
    try:
        title = oscarData['film'][i]
        year = oscarData['year_film'][i]
    
        for j in range(0, i):
            try:
                if title == oscarData['film'][j]:
                    enter = 0
                    title = ""
                    year = 0
            except:
                continue
    
        if enter != 0:
            if oscarData['winner'][i] == True:
                wins += 1
    
            for k in range(i+1, 9892):
                try:
                    if oscarData['film'][k] == title and oscarData['year_film'][k] == year:
                        if oscarData['winner'][k] == True:
                            wins += 1
                except:
                    continue
        
            for m in range(0, 9892):
                try:
                    if oscarData['film'][m] == title and oscarData['year_film'][m] == year:
                        oscarData.at[m, 'Number of wins'] = wins
                except:
                    continue
        
            wins = 0
        enter = 1
    except:
        continue

###### The new Oscar Award dataset:

In [23]:
oscarData.head()

,year_film,year_ceremony,ceremony,category,name,film,winner,Number of wins
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False,0
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True,1
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False,0
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True,3
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,0


In [24]:
oscarData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9892 entries, 0 to 10390
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_film       9892 non-null   int64 
 1   year_ceremony   9892 non-null   int64 
 2   ceremony        9892 non-null   int64 
 3   category        9892 non-null   object
 4   name            9892 non-null   object
 5   film            9892 non-null   object
 6   winner          9892 non-null   bool  
 7   Number of wins  9892 non-null   int64 
dtypes: bool(1), int64(4), object(3)
memory usage: 885.9+ KB


###### Removing duplicates:

In [25]:
for i in range(0, 9892):
    try:
        title = oscarData['film'][i]
        year = oscarData['year_film'][i]
    
        for j in range(i+1, 9892):
            try:
                if oscarData['film'][j].strip() == title.strip():
                    if str(year) == str(oscarData['year_film'][j]):
                        oscarData = oscarData.drop(j)
            except:
                continue

    except:
        continue

In [26]:
oscarData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5154 entries, 0 to 10390
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_film       5154 non-null   int64 
 1   year_ceremony   5154 non-null   int64 
 2   ceremony        5154 non-null   int64 
 3   category        5154 non-null   object
 4   name            5154 non-null   object
 5   film            5154 non-null   object
 6   winner          5154 non-null   bool  
 7   Number of wins  5154 non-null   int64 
dtypes: bool(1), int64(4), object(3)
memory usage: 456.2+ KB


### Adding variables into the Oscar Award dataset

Now that we have our cleaned Oscar Award dataset, we can add in the variables we will be using to predict how many times the film can win at the Oscars.
<p>
This will be done through data extraction from each film's IMDB page by searching for the links to each page and storing it in the dataset.

In [27]:
oscarData = oscarData.reindex()
oscarData

,year_film,year_ceremony,ceremony,category,name,film,winner,Number of wins
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False,0
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True,1
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False,0
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True,3
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,0
...,...,...,...,...,...,...,...,...
10386,2019,2020,92,WRITING (Original Screenplay),Written by Rian Johnson,Knives Out,False,0
10387,2019,2020,92,WRITING (Original Screenplay),Written by Noah Baumbach,Marriage Story,False,0
10388,2019,2020,92,WRITING (Original Screenplay),Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,0
10389,2019,2020,92,WRITING (Original Screenplay),Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,0


In [28]:
links = []
title = oscarData['film']
year = oscarData['year_film']

for n in range(0, 5154):
    try:
        x = re.sub(' ', '+', title[n])
        search = 'https://www.imdb.com/find?s=tt&q=' + x + '&ref_=nv_sr_sm'
        yes = 0
        try:
            response = requests.get(search)
            soup = BeautifulSoup(response.text, 'lxml')
            m = soup.find_all('td', {'class':'result_text'})
            for y in range(0, len(m)):
                split = m[y].text.split('(')
                try:
                    split2 = split[1].split(')')
                    split2 = split2[0].rstrip()
                    split2 = int(split2)
                except:
                    continue
                if (split2 == oscarData['year_film'][n]) | (split2 == oscarData['year_film'][n]+1):
                    links.append(soup.select('td.result_text a')[y].attrs.get('href'))
                    yes = 1
                    break
                else:
                    continue
            
            if yes != 1:
                links.append("NIL")
        except:
            links.append("NIL")
            continue
    except:
        links.append("NIL")
        continue

In [29]:
links

['/title/tt0019217/',
 '/title/tt0019071/',
 '/title/tt0018389/',
 '/title/tt0018379/',
 '/title/tt0019344/',
 '/title/tt0018455/',
 '/title/tt0017822/',
 'NIL',
 '/title/tt0017806/',
 'NIL',
 'NIL',
 '/title/tt0018515/',
 '/title/tt0019412/',
 'NIL',
 '/title/tt0018429/',
 '/title/tt0018806/',
 'NIL',
 'NIL',
 'NIL',
 '/title/tt0019304/',
 'NIL',
 '/title/tt0018578/',
 'NIL',
 'NIL',
 '/title/tt0017743/',
 '/title/tt0018037/',
 '/title/tt0018945/',
 'NIL',
 'NIL',
 '/title/tt0018526/',
 'NIL',
 'NIL',
 'NIL',
 'NIL',
 'NIL',
 '/title/tt0020499/',
 '/title/tt0020018/',
 '/title/tt0019630/',
 '/title/tt0020543/',
 '/title/tt0019257/',
 '/title/tt0020126/',
 '/title/tt0018674/',
 '/title/tt0020092/',
 '/title/tt0019824/',
 '/title/tt0019729/',
 '/title/tt0019788/',
 'NIL',
 '/title/tt0019722/',
 '/title/tt0019843/',
 '/title/tt0019630/',
 '/title/tt0019429/',
 '/title/tt0019237/',
 '/title/tt0019574/',
 'NIL',
 '/title/tt0018907/',
 'NIL',
 'NIL',
 'NIL',
 'NIL',
 'NIL',
 '/title/tt00198

In [30]:
oscarData.insert(8, 'Link', links)
oscarData

,year_film,year_ceremony,ceremony,category,name,film,winner,Number of wins,Link
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False,0,/title/tt0019217/
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True,1,/title/tt0019071/
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False,0,/title/tt0018389/
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True,3,/title/tt0018379/
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,0,/title/tt0019344/
...,...,...,...,...,...,...,...,...,...
10386,2019,2020,92,WRITING (Original Screenplay),Written by Rian Johnson,Knives Out,False,0,/title/tt0069946/
10387,2019,2020,92,WRITING (Original Screenplay),Written by Noah Baumbach,Marriage Story,False,0,/title/tt0070511/
10388,2019,2020,92,WRITING (Original Screenplay),Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,0,NIL
10389,2019,2020,92,WRITING (Original Screenplay),Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,0,NIL


We will now use the links we generated to search for each film's information on the IMDB website and storing it into our dataset. Any missing information will be labelled as NIL.

In [33]:
filecsv = open('imdbData.csv', 'w', newline='', encoding='utf8')
csv_columns = ['Title', 'Year', 'Genre', 'Rating', 'Runtime', 'Sales', 'Language', 'Age Rating', 'Director', 'Cast1', 'Cast2', 'Cast3', 'Country of Origin', 'Number of Wins']
writer = csv.DictWriter(filecsv, fieldnames=csv_columns)
writer.writeheader()

x = 1
s = requests.Session()


for q in range(0, len(oscarData)):
        

        if links[q]=='NIL':
            writer.writerow({'Title':'NIL', 'Year':'NIL', 'Genre':'NIL', 'Rating':'NIL', 'Runtime':'NIL', 'Sales':'NIL', 'Language':'NIL', 'Age Rating':'NIL', 'Director':'NIL', 'Cast1':'NIL', 'Cast2':'NA', 'Cast3':'NIL', 'Country of Origin':'NIL', 'Number of Wins':'NIL'})
            continue
        
        first_link = links[q]

        try:
            r = requests.get('http://imdb.com'+first_link)
        except requests.exceptions.ConnectionError:
            #r.status_code = "Connection refused"
            x += 1
            writer.writerow({'Title':oscarData['film'][q], 'Year':'NIL', 'Genre':'NIL', 'Rating':'NIL', 'Runtime':'NIL', 'Sales':'NIL', 'Language':'NIL', 'Age Rating':'NIL', 'Director':'NIL', 'Cast1':'NIL', 'Cast2':'NA', 'Cast3':'NIL', 'Country of Origin':'NIL', 'Number of Wins':'NIL'})
            continue

        
        soup = BeautifulSoup(r.content, 'html.parser')
        name = oscarData['film'][q]
        year = oscarData['year_film'][q]
        win = oscarData['Number of wins'][q]
        
        genre = ""
        genres = soup.find('li', {'data-testid':'storyline-genres'})
        if genres:
            genres2 = genres.find_all('a', href=re.compile("genres"))
            genre = genres2[0].contents[0]
            if len(genres2)>1:
                for g in range(1, len(genres)):
                    genre = genre + ', '
                    genre = genre + genres2[g].contents[0]
        else:
            genre = 'NIL'

            
        ratings = soup.find('span', {'class':'sc-7ab21ed2-1 jGRxWM'})
        if ratings:
            rating = ratings.contents[0]
        else:
            rating = 'NIL'
        
        
        runtimes = soup.find('li', {'data-testid':'title-techspec_runtime'})
        if runtimes:
            runtime = runtimes.find('div').contents[0]
            for r in range(1, len(runtimes.find('div').contents)):
                runtime = runtime + runtimes.find('div').contents[r]
        else:
            runtime = 'NIL'
            
        
        salesBox = soup.find('li', {'data-testid':'title-boxoffice-cumulativeworldwidegross'})
        if salesBox:
            sales = salesBox.find('span', {'class':'ipc-metadata-list-item__list-content-item'}).contents[0]
        else:
            sales = 'NIL'
            
        langs = soup.find_all(href=re.compile("language"))
        if langs:
            lang = ""
            lang = str(langs[0].contents[0])
            if len(langs)>1:
                for lg in range(1, len(langs)):
                    lang = lang + ', '
                    lang = lang + str(langs[lg].contents[0])
        else:
            genre = 'NIL'
        
        certificates = soup.find('a', href=re.compile("parental"))
        
        if certificates:
            age = certificates.contents[0]
        else:
            age = 'NIL'
            
        crew = soup.find_all('li', {'data-testid':'title-pc-principal-credit'})
        if crew:
            director = crew[0].find('a', {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).contents[0]
            try:
                cast = crew[2].find_all('a', {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
            except:
                continue
            c1 = ""
            c2 = ""
            c3 = ""
            try:
                c1 = cast[0].contents[0]
            except:
                c1 = 'NIL'
                c2 = 'NIL'
                c3 = 'NIL'
            try:
                c2 = cast[1].contents[0]
            except:
                c2 = 'NIL'
                c3 = 'NIL'
            try:
                c3 = cast[2].contents[0]
            except:
                c3 = 'NIL'
                
        else:
            director = 'NIL'
            c1 = 'NIL'
            c2 = 'NIL'
            c3 = 'NIL'
            
        countries = soup.find_all('a', href=re.compile("country_of_origin"))
        if countries:
            country = ""
            country = countries[0].contents[0]
            if len(countries)>1:
                for c in range(1, len(countries)):
                    country = country + ', '
                    country = country + countries[c].contents[0]
        else:
            country = 'NIL'

            
        
        writer.writerow({'Title':name, 'Year':year, 'Genre':genre, 'Rating':rating, 'Runtime':runtime, 'Sales':sales, 'Language':lang, 'Age Rating':age, 'Director':director, 'Cast1':c1, 'Cast2':c2, 'Cast3':c3, 'Country of Origin':country, 'Number of Wins':win})
        
filecsv.close()

###### Our new Oscar Award dataset with the number of wins and the information for each predictor:

In [34]:
oscarData = pd.read_csv('imdbData.csv')
oscarData.head()

,Title,Year,Genre,Rating,Runtime,Sales,Language,Age Rating,Director,Cast1,Cast2,Cast3,Country of Origin,Number of Wins
0,The Noose,1927,Drama,6.9,1 hour 5 minutes,NIL,"None, English",Add content advisory,John Francis Dillon,Richard Barthelmess,Montagu Love,Robert Emmett O'Connor,United States,0
1,The Last Command,1927,"Drama, History",8.0,1 hour 28 minutes,NIL,"None, English",Not Rated,Josef von Sternberg,Emil Jannings,Evelyn Brent,William Powell,United States,1
2,A Ship Comes In,1927,Drama,5.6,1 hour 10 minutes,NIL,None,Add content advisory,William K. Howard,Rudolph Schildkraut,Louise Dresser,Milton Holmes,United States,0
3,7th Heaven,1927,"Drama, Romance",7.6,1 hour 50 minutes,NIL,"None, English",Add content advisory,Frank Borzage,Janet Gaynor,Charles Farrell,Ben Bard,United States,3
4,Sadie Thompson,1927,Drama,7.2,1 hour 37 minutes,NIL,English,Unrated,Raoul Walsh,Lionel Barrymore,Blanche Friderici,Charles Lane,United States,0


### Cleaning the dataset

Now, as we require all the information on the predictors for each film, we need to remove those films that are missing the relevant predictors, in this case, those that have 'NIL' values.

In [35]:
for n in range(0, len(oscarData)):
    if (oscarData['Year'][n]=='NIL' or oscarData['Sales'][n]=='NIL' or oscarData['Age Rating'][n]=='Add content advisory' or oscarData['Age Rating'][n]=='Parents guide' or oscarData['Cast3'][n]=='NIL' or oscarData['Cast2'][n]=='NIL' or oscarData['Cast1'][n]=='NIL' or oscarData['Director'][n]=='NIL'):
        oscarData = oscarData.drop(index = n)

For our "Runtime" predictor, we want to standardise the timing into minutes only as it is a numerical variable.

In [39]:
# Storing runtime as a new dataframe
runtime = pd.DataFrame(oscarData['Runtime'])
runtime.head()

,Runtime
5,1 hour 34 minutes
21,2 hours 24 minutes
65,2 hours 10 minutes
120,1 hour 32 minutes
157,1 hour 23 minutes


In [40]:
#inserting the necessary columns
runtime.insert(1, "Hours", 0)
runtime.insert(2, "Minutes", 0)

In [42]:
for i in range(0, len(runtime)):
    try:
        runtime["Runtime"][i]
    except:
        continue
    if "hours" in runtime["Runtime"][i]:
        temp1 = runtime["Runtime"][i].split("hours")
        hour = temp1[0].strip()
        if "minutes" in runtime["Runtime"][i]:
            temp2 = temp1[1].split("minutes")
            minute = temp2[0].strip()
        elif "minute" in runtime["Runtime"][i]:
            temp2 = temp1[1].split("minute")
            minute = temp2[0].strip()
        
    elif "hour" in runtime["Runtime"][i]:
        temp1 = runtime["Runtime"][i].split("hour")
        hour = temp1[0].strip()
        if "minutes" in runtime["Runtime"][i]:
            temp2 = temp1[1].split("minutes")
            minute = temp2[0].strip()
        elif "minute" in runtime["Runtime"][i]:
            temp2 = temp1[1].split("minute")
            minute = temp2[0].strip()
    else:
        if "minutes" in runtime["Runtime"][i]:
            temp1 = runtime["Runtime"][i].split("minutes")
            hour = 0
            minute = temp1[0].strip()
        
        elif "minute" in runtime["Runtime"][i]:
            temp1 = runtime["Runtime"][i].split("minute")
            hour = 0
            minute = temp1[0].strip()
        
    runtime.at[i,'Hours'] = hour
    runtime.at[i,'Minutes'] = minute

In [43]:
#new runtime dataframe
runtime.head()

,Runtime,Hours,Minutes
5,1 hour 34 minutes,1,34
21,2 hours 24 minutes,2,24
65,2 hours 10 minutes,2,10
120,1 hour 32 minutes,1,32
157,1 hour 23 minutes,1,23


In [45]:
#converting the hours into minutes and putting back the data into the oscarData dataframe
time = 0
for i in range(0, len(runtime)):
    try:
        runtime["Hours"][i]
    except:
        continue
    time = int(runtime["Hours"][i])*60 + int(runtime["Minutes"][i])
    oscarData.at[i,'Runtime'] = time

###### Cleaned Oscar Award dataset:

In [46]:
oscarData.head()

,Title,Year,Genre,Rating,Runtime,Sales,Language,Age Rating,Director,Cast1,Cast2,Cast3,Country of Origin,Number of Wins
5,Sunrise,1927,"Drama, Romance",8.1,94,"$121,107","None, English",Passed,F.W. Murnau,George O'Brien,Janet Gaynor,Margaret Livingston,United States,3
21,Wings,1927,"Drama, Romance",7.6,144,$746,English,PG-13,William A. Wellman,Clara Bow,Charles 'Buddy' Rogers,Richard Arlen,United States,1
65,Hollywood Revue,1928,"Comedy, Music",5.8,130,"$5,277,780",English,Passed,Charles Reisner,Conrad Nagel,Jack Benny,John Gilbert,United States,0
120,Morocco,1930,"Drama, Romance",7.0,92,$191,"English, French, Spanish, Arabic, Italian",Passed,Josef von Sternberg,Gary Cooper,Marlene Dietrich,Adolphe Menjou,United States,0
157,The Public Enemy,1930,"Crime, Drama",7.6,83,"$1,214,260",English,Passed,William A. Wellman,James Cagney,Jean Harlow,Edward Woods,United States,0


## Adding our final predictors

Finally, once we have all our information, we are left with adding the last 2 predictors - Director Rank and Sum of Cast Rankings. 
<p>
We will be using the Top 50 Directors list obtained from IMDB to add the rankings for each director in each film and the Top 1000 Actors and Actresses list also obtained from IMDB to add the rankings for each cast in each film.

### Obtaining the datasets for:
> 1. **Top 50 Directors** 
> 2. **Top 1000 Actors and Actresses**
------------------------

###### Top 50 Directors dataset:

In [49]:
# The dataset is in CSV format, hence we use the read_csv function from Pandas.
# Immediately after importing, we will take a quick look at the data using the head function.
directorData = pd.read_csv('Top 50 directors.csv', encoding = 'ISO-8859-1')
directorData.head()

,Rank,Name of Director
0,1,Steven Spielberg
1,2,Martin Scorsese
2,3,Francis Ford Coppola
3,4,Stanley Kubrick
4,5,Alfred Hitchcock


Check the vital statistics of the dataset using the `type` and `shape` attributes.

In [50]:
print("Data type : ", type(directorData))
print("Data dims : ", directorData.shape)

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (50, 2)


Check the variables (and their types) in the dataset using the `dtypes` attribute.

In [51]:
print(directorData.dtypes)

Rank                 int64
Name of Director    object
dtype: object


###### Top 1000 Actors and Actresses dataset:¶

In [52]:
# The dataset is in CSV format, hence we use the read_csv function from Pandas.
# Immediately after importing, we will take a quick look at the data using the head function.
actorData = pd.read_csv('Top 1000 Actors and Actresses.csv')
actorData.head()

,Position,Const,Created,Modified,Description,Name,Known For,Birth Date
0,1,nm0000134,2014-03-09,2014-03-09,NaN,Robert De Niro,Raging Bull,1943-08-17
1,2,nm0000197,2014-03-09,2015-10-25,NaN,Jack Nicholson,Chinatown,1937-04-22
2,3,nm0000008,2014-03-09,2014-03-09,NaN,Marlon Brando,Apocalypse Now,1924-04-03
3,4,nm0000243,2014-03-09,2014-03-09,NaN,Denzel Washington,Fences,1954-12-28
4,5,nm0000031,2014-03-09,2014-03-09,NaN,Katharine Hepburn,The Lion in Winter,1907-05-12


Check the vital statistics of the dataset using the `type` and `shape` attributes.

In [53]:
print("Data type : ", type(actorData))
print("Data dims : ", actorData.shape)

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (1000, 8)


Check the variables (and their types) in the dataset using the `dtypes` attribute.

In [54]:
print(actorData.dtypes)

Position         int64
Const           object
Created         object
Modified        object
Description    float64
Name            object
Known For       object
Birth Date      object
dtype: object


### Adding our variables to the dataset

As we are using the actors' and directors' popularity to determine the number of times a film wins at the Oscar Awards, we need to add these variables to the dataset using the existing actor and directors names.
<p>
We begin by adding columns into the oscarData dataframe.

###### Adding the necessary columns:

This includes:
> 1. Cast1 Ranking
> 2. Cast2 Ranking
> 3. Cast3 Ranking
> 4. Director Ranking
> 5. Sum of Rankings

The default value of director ranks is set as 100000. This allows the rankings of the top 50 Directors to weigh more.

In [55]:
oscarData.insert(9, "Director Rank", 100000)

The default value of cast ranks is also set as 100000. This allows the rankings of the top 1000 Actors and Actresses to weigh more. Henceforth, the default value of sum of cast rankings is set at 400000, which is the sum of default value of all the cast ranks.

In [56]:
oscarData.insert(11, "Cast1 Rank", 100000)
oscarData.insert(13, "Cast2 Rank", 100000)
oscarData.insert(15, "Cast3 Rank", 100000)
oscarData.insert(16, "Sum of Cast Rankings", 400000)

###### The new dataframe:

In [57]:
oscarData.head()

,Title,Year,Genre,Rating,Runtime,Sales,Language,Age Rating,Director,Director Rank,Cast1,Cast1 Rank,Cast2,Cast2 Rank,Cast3,Cast3 Rank,Sum of Cast Rankings,Country of Origin,Number of Wins
5,Sunrise,1927,"Drama, Romance",8.1,94,"$121,107","None, English",Passed,F.W. Murnau,100000,George O'Brien,100000,Janet Gaynor,100000,Margaret Livingston,100000,400000,United States,3
21,Wings,1927,"Drama, Romance",7.6,144,$746,English,PG-13,William A. Wellman,100000,Clara Bow,100000,Charles 'Buddy' Rogers,100000,Richard Arlen,100000,400000,United States,1
65,Hollywood Revue,1928,"Comedy, Music",5.8,130,"$5,277,780",English,Passed,Charles Reisner,100000,Conrad Nagel,100000,Jack Benny,100000,John Gilbert,100000,400000,United States,0
120,Morocco,1930,"Drama, Romance",7.0,92,$191,"English, French, Spanish, Arabic, Italian",Passed,Josef von Sternberg,100000,Gary Cooper,100000,Marlene Dietrich,100000,Adolphe Menjou,100000,400000,United States,0
157,The Public Enemy,1930,"Crime, Drama",7.6,83,"$1,214,260",English,Passed,William A. Wellman,100000,James Cagney,100000,Jean Harlow,100000,Edward Woods,100000,400000,United States,0


Now, we will add the corresponding director and cast ranks.

###### Adding Director Ranks:

In [60]:
#Adding the director ranks by comparing the name of the director in the oscarData dataframe and that in our directorData dataframe.
#Those that do not have a rank in the directorData dataframe will be assigned a rank of 100000.
#This places more weight on those directors who have a rank.
yes = 0
for i in range(0,len(oscarData)):
    try:
        oscarData['Name of Director'][i]
    except:
        continue
    for j in range(0,len(directorData)):
        try:
            oscarData['Director'][j]
        except:
            continue
        if(oscarData['Director'][i] == directorData['Name of Director'][j]):
            oscarData.at[i,'Director Rank'] = directorData['Rank'][j]
            yes = 1
    if yes != 1:
        oscarData.at[i,'Director Rank'] = '100000'
    yes = 0

###### Adding Cast Ranks:

In [61]:
#Adding the cast ranks by comparing the name of the cast in the oscarData dataframe and that in our directorData dataframe.
#Those that do not have a rank in the directorData dataframe will be assigned a rank of 100000.
#This places more weight on those casts who have a rank.
yes1 = 0
yes2 = 0
yes3 = 0
for i in range(0,len(oscarData)):
    try:
        oscarData['Cast1'][i]
    except:
        continue
    for j in range(0,len(actorData)):
        try:
            oscarData['Cast1'][i]
        except:
            continue
        if(oscarData['Cast1'][i].strip() == actorData['Name'][j]):
            oscarData.at[i,'Cast1 Rank'] = actorData['Position'][j]
            yes1 = 1
            
        if(oscarData['Cast2'][i].strip() == actorData['Name'][j]):
            oscarData.at[i,'Cast2 Rank'] = actorData['Position'][j]
            yes2 = 1
            
        if(oscarData['Cast3'][i].strip() == actorData['Name'][j]):
            oscarData.at[i,'Cast3 Rank'] = actorData['Position'][j]
            yes3 = 1
    #if there was no corresponding rank data, the rank remains at 100000        
    if yes1 != 1:
        oscarData.at[i,'Cast1 Rank'] = '100000'
        
    if yes2 != 1:
        oscarData.at[i,'Cast2 Rank'] = '100000'
        
    if yes3 != 1:
        oscarData.at[i,'Cast3 Rank'] = '100000'
    #initialize all yes variables back to 0
    yes1 = 0
    yes2 = 0
    yes3 = 0

###### Adding Sum of Cast Ranks:

In [65]:
#Adding the Sum of Cast Rankings
sum = 0

for i in range(0,len(oscarData)):
    try:
        oscarData['Cast1 Rank'][i]
    except:
        continue
    sum += int(oscarData['Cast1 Rank'][i]) + int(oscarData['Cast2 Rank'][i]) + int(oscarData['Cast3 Rank'][i])
    oscarData.at[i,'Sum of Cast Rankings'] = sum
    sum = 0

###### The new oscarData dataframe:

In [66]:
oscarData.head()

,Title,Year,Genre,Rating,Runtime,Sales,Language,Age Rating,Director,Director Rank,Cast1,Cast1 Rank,Cast2,Cast2 Rank,Cast3,Cast3 Rank,Sum of Cast Rankings,Country of Origin,Number of Wins
5,Sunrise,1927,"Drama, Romance",8.1,94,"$121,107","None, English",Passed,F.W. Murnau,100000,George O'Brien,100000,Janet Gaynor,387,Margaret Livingston,100000,200387,United States,3
21,Wings,1927,"Drama, Romance",7.6,144,$746,English,PG-13,William A. Wellman,100000,Clara Bow,100000,Charles 'Buddy' Rogers,100000,Richard Arlen,100000,300000,United States,1
65,Hollywood Revue,1928,"Comedy, Music",5.8,130,"$5,277,780",English,Passed,Charles Reisner,100000,Conrad Nagel,100000,Jack Benny,100000,John Gilbert,100000,300000,United States,0
120,Morocco,1930,"Drama, Romance",7.0,92,$191,"English, French, Spanish, Arabic, Italian",Passed,Josef von Sternberg,100000,Gary Cooper,71,Marlene Dietrich,241,Adolphe Menjou,100000,100312,United States,0
157,The Public Enemy,1930,"Crime, Drama",7.6,83,"$1,214,260",English,Passed,William A. Wellman,100000,James Cagney,40,Jean Harlow,244,Edward Woods,100000,100284,United States,0


In [67]:
oscarData.to_csv("fullOscarData.csv")

In [68]:
#this is a utility function for the Exploratory Data Analysis portion
oscarData.to_csv("fullOscarData1.csv")